In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
test = pd.read_csv('test.csv')

In [3]:
train = pd.read_csv('train.csv')

In [4]:
sample_submission = pd.read_csv('sample_submission.csv')

In [5]:
train.head()

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...


In [6]:
test.head()

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...


In [7]:
sample_submission.head()

,oid,category
0,1622114,athletics
1,1663817,autosport
2,3174332,basketball
3,3469228,extreme
4,3905302,boardgames


In [8]:
target = train['category']

In [9]:
X_train = train.drop(['category'], axis=1)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
X_train['text'] = X_train['text'].str.lower()

In [12]:
test['text'] = test['text'].str.lower()

In [13]:
test = test.set_index('oid')

In [14]:
X_train = X_train.set_index('oid')

In [15]:
cv = CountVectorizer()
cv.fit(X_train['text'])

len(cv.vocabulary_)

146608

In [16]:
print(cv.get_feature_names()[:50])

['00', '000', '0000', '00000', '000000', '0000000', '00000000', '000000000', '0000000000', '00000000000', '000000000000000', '000v', '000м', '000р', '000т', '000ға', '002', '003', '004', '005', '0067soft3', '007', '0089', '00tokenotokenoid', '00zed00', '00дорогие', '00мин', '00мск', '01', '010', '013', '014', '01412metokenoid', '016', '01c', '01м', '02', '021', '0233610', '028', '03', '030', '032', '033', '035', '036', '037', '039', '03e3a5693e8553tokenoid', '04']


C:\Users\Александр\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
X_train_transform = cv.transform(X_train['text'])
X_train_transform

<38740x146608 sparse matrix of type '<class 'numpy.int64'>'
	with 1732919 stored elements in Compressed Sparse Row format>

In [18]:
test_transform = cv.transform(test['text'])
test_transform

<26260x146608 sparse matrix of type '<class 'numpy.int64'>'
	with 1131935 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.linear_model import LogisticRegression

In [142]:
logit = LogisticRegression(solver="lbfgs", n_jobs=-1, random_state=7, C=1)
logit.fit(X_train_transform, target)

LogisticRegression(C=1, n_jobs=-1, random_state=7)

In [143]:
round(logit.score(X_train_transform, target), 3)

0.993

In [22]:
import matplotlib.pyplot as plt

In [144]:
probs = pd.DataFrame(logit.predict_proba(test_transform))

In [145]:
probs.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.013766,0.052918,0.016213,0.002353,0.684457,0.023909,0.084320,0.012840,0.045613,0.027505,0.009173,0.012453,0.014480
1,0.012474,0.168140,0.000516,0.000232,0.139236,0.002723,0.084501,0.000653,0.074190,0.000479,0.000321,0.510484,0.006053
2,0.001407,0.011981,0.010934,0.026579,0.121162,0.070080,0.003818,0.005067,0.010340,0.035960,0.679994,0.002580,0.020096
3,0.000821,0.005240,0.000011,0.001975,0.948601,0.040315,0.000052,0.000004,0.000160,0.000200,0.001804,0.000815,0.000003
4,0.001308,0.013044,0.001949,0.001488,0.001777,0.006703,0.007806,0.004439,0.010178,0.001486,0.935537,0.001790,0.012494


In [146]:
values = pd.DataFrame([max(i) for i in probs.values])


In [147]:
values.head()

,0
0,0.684457
1,0.510484
2,0.679994
3,0.948601
4,0.935537


In [148]:
values.shape[0]

26260

In [149]:
values[0].value_counts().sort_index().head(20)

0.108008    1
0.110799    1
0.112901    1
0.114078    1
0.114115    1
0.115591    1
0.118710    1
0.119535    1
0.122179    1
0.122901    1
0.124482    1
0.125091    1
0.125354    1
0.127156    1
0.127284    1
0.127668    1
0.127804    1
0.128368    1
0.128709    1
0.128927    1
Name: 0, dtype: int64

In [150]:
pred = pd.DataFrame(logit.predict(test_transform))

In [151]:
pred.head()

,0
0,esport
1,volleyball
2,tennis
3,esport
4,tennis


In [152]:
test.head()

,oid,category,prob
0,749208109,esport,0.572546
1,452466036,volleyball,0.475558
2,161038103,tennis,0.474782
3,663621910,esport,0.900373
4,566255305,tennis,0.835611


In [153]:
test = test.reset_index()

In [157]:
test.set_index('index')

,oid,category,prob
index,,,
0,749208109,esport,0.572546
1,452466036,volleyball,0.475558
2,161038103,tennis,0.474782
3,663621910,esport,0.900373
4,566255305,tennis,0.835611
...,...,...,...
26255,169728316,boardgames,0.999463
26256,279369911,volleyball,0.990143
26257,600699419,boardgames,0.988438


In [158]:
test['category'] = pred

In [159]:
test['prob'] = values[0]

In [160]:
test.head()

,index,oid,category,prob
0,0,749208109,esport,0.684457
1,1,452466036,volleyball,0.510484
2,2,161038103,tennis,0.679994
3,3,663621910,esport,0.948601
4,4,566255305,tennis,0.935537


In [37]:
test = test.drop('text', axis=1)

In [161]:
test.sort_values(by='prob').head()

,index,oid,category,prob
9491,9491,543463128,volleyball,0.108008
439,439,126999231,basketball,0.110799
17960,17960,627942312,motosport,0.112901
20099,20099,916932407,athletics,0.114078
17758,17758,655435538,football,0.114115


In [162]:
test.shape[0]

26260

In [39]:
test2 = test.copy()

In [40]:
test2.head()

,oid,category,prob
0,749208109,esport,0.572546
1,452466036,volleyball,0.475558
2,161038103,tennis,0.474782
3,663621910,esport,0.900373
4,566255305,tennis,0.835611


In [54]:
test2.loc[test2['prob'] <= 0.2, 'category'] = np.nan

In [55]:
test2.sort_values(by='prob')

,oid,category,prob
439,126999231,NaN,0.101631
12153,307726713,NaN,0.105692
17960,627942312,NaN,0.107497
480,543463128,NaN,0.110518
9702,805173438,NaN,0.111410
...,...,...,...
24870,191775103,basketball,1.000000
12491,821416534,hockey,1.000000
24823,982297104,autosport,1.000000
21945,581191034,athletics,1.000000


In [163]:
test3 = test.copy()

In [164]:
test3 = test[test['prob'] > 0.7]

In [165]:
test3.head()

,index,oid,category,prob
3,3,663621910,esport,0.948601
4,4,566255305,tennis,0.935537
5,5,332983824,volleyball,0.825545
7,7,770457906,volleyball,1.000000
8,8,720955404,boardgames,0.982327


In [166]:
test3.shape[0]

18604

In [167]:
test3.sort_values(by='prob').head()

,index,oid,category,prob
9773,9773,331906829,hockey,0.700017
21262,21262,454563234,boardgames,0.700207
7448,7448,805934915,hockey,0.700381
3664,3664,802544922,martial_arts,0.700483
9417,9417,720267537,winter_sport,0.700514


In [168]:
final = test3.drop('prob', axis=1)

In [169]:
final.isnull().sum()

index       0
oid         0
category    0
dtype: int64

In [170]:
final.head()

,index,oid,category
3,3,663621910,esport
4,4,566255305,tennis
5,5,332983824,volleyball
7,7,770457906,volleyball
8,8,720955404,boardgames


In [171]:
final = final.drop('index', axis=1)

In [172]:
final.head()

,oid,category
3,663621910,esport
4,566255305,tennis
5,332983824,volleyball
7,770457906,volleyball
8,720955404,boardgames


In [173]:
submis = final.copy()

In [174]:
submis.head()

,oid,category
3,663621910,esport
4,566255305,tennis
5,332983824,volleyball
7,770457906,volleyball
8,720955404,boardgames


In [176]:
test4 = test[test['prob'] > 0.65]

In [178]:
c

In [179]:
test4.head()

,oid,category
0,749208109,esport
2,161038103,tennis
3,663621910,esport
4,566255305,tennis
5,332983824,volleyball


In [180]:
test4.to_csv('submis_065.csv')

In [181]:
test5 = test[test['prob'] > 0.67]

In [182]:
test5 = test5.drop(['prob', 'index'], axis=1)

In [183]:
test5.to_csv('submis_067.csv')

In [186]:
test6 = test[test['prob'] > 0.725]

In [187]:
test6 = test6.drop(['prob', 'index'], axis=1)

In [188]:
test6.to_csv('submis_0725.csv')

In [189]:
test7 = test[test['prob'] > 0.74]

In [190]:
test7 = test7.drop(['prob', 'index'], axis=1)

In [ ]:
test7.to_csv('submis_072.csv')